In [1]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

C:\Users\BigJulius\AppData\Local\Temp\ipykernel_16964\3255632492.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
pd.options.display.max_columns = 100

In [3]:
username = 'root'
password='root'
host='localhost'
database='jmdb'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [4]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [5]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [6]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Etiquettes', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [7]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [8]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                                              Titre  \
0    3721  The Lustful Shogun and his 21 Concubines (vostfr)   
1    3720                                      Les Indomptés   
2    3719                                   Le Deuxième Acte   
3    3718                                  Le Bouc émissaire   
4    3717                                    Gothic (vostfr)   

                 Titre alternatif Année  Durée Chemin de l'affiche  \
0   エロ将軍と二十一人の愛妾 (Titre original)  1972     95          003805.jpg   
1       Mobsters (Titre original)  1991    104          003804.jpg   
2                             NaN  2024     85          003803.jpg   
3  The Scapegoat (Titre original)  1959     92          003802.jpg   
4                             NaN  1986     84          003801.jpg   

                         Bande-annonce  \
0                                  NaN   
1  www.youtube.com/watch?v=IVRHM-BHd4A   
2  www.youtube.com/watch?v=7aiYYA7o6_0   
3  www.youtube.com/watch?v

In [9]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [10]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [11]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [12]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [13]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [14]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [15]:
display(df_director)

,id,Réalisateur(s)
0,1,Norifumi Suzuki
1,2,Michael Karbelnikoff
2,3,Quentin Dupieux
3,4,Robert Hamer
4,5,Ken Russell
...,...,...
1825,1826,Richard C. Sarafian
1826,1827,Chris Petit
1827,1828,Brett Ratner
1828,1829,Buster Keaton


In [16]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3721,The Lustful Shogun and his 21 Concubines (vostfr),エロ将軍と二十一人の愛妾 (Titre original),1972,95,003805.jpg,NaN,NaN,Le 5 octobre 1773 naissent au même moment Toyo...,F:\#DIVX\DIVX MOVIEZ\DRAMA\The Lustful Shogun ...,Matroska (MKV),NaN,Fichier multimédia,1.74 GB,movie/126896,2098680.0
1,3720,Les Indomptés,Mobsters (Titre original),1991,104,003804.jpg,www.youtube.com/watch?v=IVRHM-BHd4A,Ils n'ont pas reçu d'ordres... Ils ont pris la...,"Dans les années 1910, deux gangs du milieu new...",F:\#DIVX\DIVX MOVIEZ\POLAR\Les Indomptés [UNCU...,Matroska (MKV),NaN,Fichier multimédia,2.57 GB,movie/21219,102460.0
2,3719,Le Deuxième Acte,NaN,2024,85,003803.jpg,www.youtube.com/watch?v=7aiYYA7o6_0,NaN,"Florence veut présenter David, l'homme dont el...",F:\#DIVX\DIVX MOVIEZ\COMEDIE\Le Deuxieme Acte ...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/1161108,30784256.0
3,3718,Le Bouc émissaire,The Scapegoat (Titre original),1959,92,003802.jpg,www.youtube.com/watch?v=knHLALAknFo,He took another man's name... lived another ma...,"Un maître d'école anglais rencontre son sosie,...",F:\#DIVX\DIVX MOVIEZ\POLAR\Le Bouc Emissaire -...,Matroska (MKV),NaN,Fichier multimédia,2.31 GB,movie/126919,53247.0
4,3717,Gothic (vostfr),NaN,1986,84,003801.jpg,www.youtube.com/watch?v=J_JQ8o2M68w,"Conjure up your deepest, darkest fear... now c...","Le 16 juin 1816 en Suisse, la nuit la plus cel...",F:\#DIVX\DIVX MOVIEZ\MadMov\Gothic (1986) VOST...,Matroska (MKV),NaN,Fichier multimédia,1.05 GB,movie/33516,91142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,5,le mécano de la général,The General (Titre original),1926,79,000005.jpg,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3717,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,000004.jpg,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3718,3,Les feux de la rampe,Limelight (Titre original),1952,137,000003.jpg,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3719,2,Les temps modernes,Modern Times (Titre original),1936,87,000002.jpg,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [17]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3721
1,0,3564
2,0,3439
3,0,2962
4,1,3720
...,...,...
4012,1825,20
4013,1826,18
4014,1827,15
4015,1828,5


In [18]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id        Réalisateur(s)
0  1831       Norifumi Suzuki
1     2  Michael Karbelnikoff
2     3       Quentin Dupieux
3     4          Robert Hamer
4     5           Ken Russell


OperationalError: (pymysql.err.OperationalError) (3730, "Cannot drop table 'director' referenced by a foreign key constraint 'fk_movie_director_director' on table 'movie_director'.")
[SQL: 
DROP TABLE director]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [37]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [38]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [39]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3721

# table movie, screenwriter et intermediaire

In [40]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [41]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [42]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [43]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [44]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [45]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [46]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,Masahiro Kakefuda
1,1,Norifumi Suzuki
2,2,Michael Mahern
3,3,Quentin Dupieux
4,4,Daphne du Maurier
...,...,...
3990,3990,Ted Tally
3991,3991,David Veloz
3992,3992,Richard Rutowski
3993,3993,Buster Keaton


In [47]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3721,The Lustful Shogun and his 21 Concubines (vostfr),エロ将軍と二十一人の愛妾 (Titre original),1972,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le 5 octobre 1773 naissent au même moment Toyo...,F:\#DIVX\DIVX MOVIEZ\DRAMA\The Lustful Shogun ...,Matroska (MKV),NaN,Fichier multimédia,1.74 GB,movie/126896,2098680.0
1,3720,Les Indomptés,Mobsters (Titre original),1991,104,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IVRHM-BHd4A,Ils n'ont pas reçu d'ordres... Ils ont pris la...,"Dans les années 1910, deux gangs du milieu new...",F:\#DIVX\DIVX MOVIEZ\POLAR\Les Indomptés [UNCU...,Matroska (MKV),NaN,Fichier multimédia,2.57 GB,movie/21219,102460.0
2,3719,Le Deuxième Acte,NaN,2024,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=7aiYYA7o6_0,NaN,"Florence veut présenter David, l'homme dont el...",F:\#DIVX\DIVX MOVIEZ\COMEDIE\Le Deuxieme Acte ...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/1161108,30784256.0
3,3718,Le Bouc émissaire,The Scapegoat (Titre original),1959,92,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=knHLALAknFo,He took another man's name... lived another ma...,"Un maître d'école anglais rencontre son sosie,...",F:\#DIVX\DIVX MOVIEZ\POLAR\Le Bouc Emissaire -...,Matroska (MKV),NaN,Fichier multimédia,2.31 GB,movie/126919,53247.0
4,3717,Gothic (vostfr),NaN,1986,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=J_JQ8o2M68w,"Conjure up your deepest, darkest fear... now c...","Le 16 juin 1816 en Suisse, la nuit la plus cel...",F:\#DIVX\DIVX MOVIEZ\MadMov\Gothic (1986) VOST...,Matroska (MKV),NaN,Fichier multimédia,1.05 GB,movie/33516,91142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3717,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3718,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3719,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [48]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3721
1,0,3439
2,0,3245
3,0,2994
4,1,3721
...,...,...
7200,3990,15
7201,3991,14
7202,3992,14
7203,3993,5


In [49]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id      Scénariste(s)
0  3995  Masahiro Kakefuda
1     1    Norifumi Suzuki
2     2     Michael Mahern
3     3    Quentin Dupieux
4     4  Daphne du Maurier
     id  Numéro
0  3995    3721
1  3995    3439
2  3995    3245
3  3995    2994
4     1    3721


7205

In [50]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [51]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [52]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [53]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [54]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [55]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [56]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [57]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [58]:
display(df_music)

,id,Compositeur(s)
0,0,NaN
1,1,Bronislau Kaper
2,2,Thomas Dolby
3,3,Kenjirô Hirose
4,4,Ben Salisbury
...,...,...
1313,1313,Machine Head
1314,1314,Photek
1315,1315,John Frizzell
1316,1316,Leonard Cohen


In [59]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3721,The Lustful Shogun and his 21 Concubines (vostfr),エロ将軍と二十一人の愛妾 (Titre original),1972,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le 5 octobre 1773 naissent au même moment Toyo...,F:\#DIVX\DIVX MOVIEZ\DRAMA\The Lustful Shogun ...,Matroska (MKV),NaN,Fichier multimédia,1.74 GB,movie/126896,2098680.0
1,3720,Les Indomptés,Mobsters (Titre original),1991,104,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IVRHM-BHd4A,Ils n'ont pas reçu d'ordres... Ils ont pris la...,"Dans les années 1910, deux gangs du milieu new...",F:\#DIVX\DIVX MOVIEZ\POLAR\Les Indomptés [UNCU...,Matroska (MKV),NaN,Fichier multimédia,2.57 GB,movie/21219,102460.0
2,3719,Le Deuxième Acte,NaN,2024,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=7aiYYA7o6_0,NaN,"Florence veut présenter David, l'homme dont el...",F:\#DIVX\DIVX MOVIEZ\COMEDIE\Le Deuxieme Acte ...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/1161108,30784256.0
3,3718,Le Bouc émissaire,The Scapegoat (Titre original),1959,92,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=knHLALAknFo,He took another man's name... lived another ma...,"Un maître d'école anglais rencontre son sosie,...",F:\#DIVX\DIVX MOVIEZ\POLAR\Le Bouc Emissaire -...,Matroska (MKV),NaN,Fichier multimédia,2.31 GB,movie/126919,53247.0
4,3717,Gothic (vostfr),NaN,1986,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=J_JQ8o2M68w,"Conjure up your deepest, darkest fear... now c...","Le 16 juin 1816 en Suisse, la nuit la plus cel...",F:\#DIVX\DIVX MOVIEZ\MadMov\Gothic (1986) VOST...,Matroska (MKV),NaN,Fichier multimédia,1.05 GB,movie/33516,91142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3717,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3718,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3719,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [60]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3721
1,0,3720
2,0,3719
3,0,3714
4,0,3713
...,...,...
4122,1313,46
4123,1314,46
4124,1315,42
4125,1316,14


In [61]:
# Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# Si une ligne avec 'Compositeur(s)' NULL est trouvée
if not null_music_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_music.loc[null_music_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_music['id'].max()
    if max_id > 0:
        df_music.loc[df_music.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_music.head())

# Enregistrer le DataFrame modifié dans la table 'music' de la base de données
df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_music.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


     id   Compositeur(s)
0  1318              NaN
1     1  Bronislau Kaper
2     2     Thomas Dolby
3     3   Kenjirô Hirose
4     4    Ben Salisbury
     id  Numéro
0  1318    3721
1  1318    3720
2  1318    3719
3  1318    3714
4  1318    3713


4127

In [62]:
# df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

In [63]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

4127

# table movie, studio et intermediaire

In [64]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [65]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [66]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [67]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [68]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [69]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [70]:
display(df_studio)

,id,Studio
0,0,Toei Company
1,1,Universal Pictures
2,2,Chi-Fou-Mi Productions
3,3,Du Maurier-Guinness
4,4,Virgin Vision
...,...,...
997,997,ADR Distribution
998,998,A-Film Distribution
999,999,Dybex
1000,1000,British Empire Films Australia


In [71]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3721,The Lustful Shogun and his 21 Concubines (vostfr),エロ将軍と二十一人の愛妾 (Titre original),1972,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le 5 octobre 1773 naissent au même moment Toyo...,F:\#DIVX\DIVX MOVIEZ\DRAMA\The Lustful Shogun ...,Matroska (MKV),NaN,Fichier multimédia,1.74 GB,movie/126896,2098680.0
1,3720,Les Indomptés,Mobsters (Titre original),1991,104,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IVRHM-BHd4A,Ils n'ont pas reçu d'ordres... Ils ont pris la...,"Dans les années 1910, deux gangs du milieu new...",F:\#DIVX\DIVX MOVIEZ\POLAR\Les Indomptés [UNCU...,Matroska (MKV),NaN,Fichier multimédia,2.57 GB,movie/21219,102460.0
2,3719,Le Deuxième Acte,NaN,2024,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=7aiYYA7o6_0,NaN,"Florence veut présenter David, l'homme dont el...",F:\#DIVX\DIVX MOVIEZ\COMEDIE\Le Deuxieme Acte ...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/1161108,30784256.0
3,3718,Le Bouc émissaire,The Scapegoat (Titre original),1959,92,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=knHLALAknFo,He took another man's name... lived another ma...,"Un maître d'école anglais rencontre son sosie,...",F:\#DIVX\DIVX MOVIEZ\POLAR\Le Bouc Emissaire -...,Matroska (MKV),NaN,Fichier multimédia,2.31 GB,movie/126919,53247.0
4,3717,Gothic (vostfr),NaN,1986,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=J_JQ8o2M68w,"Conjure up your deepest, darkest fear... now c...","Le 16 juin 1816 en Suisse, la nuit la plus cel...",F:\#DIVX\DIVX MOVIEZ\MadMov\Gothic (1986) VOST...,Matroska (MKV),NaN,Fichier multimédia,1.05 GB,movie/33516,91142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3717,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3718,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3719,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [72]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3721
1,0,3716
2,0,3681
3,0,3639
4,0,3565
...,...,...
3719,997,182
3720,998,172
3721,999,169
3722,1000,108


In [73]:
# Trouver l'index de la ligne avec le 'Studio' NULL
null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# Si une ligne avec 'Studio' NULL est trouvée
if not null_studio_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_studio.loc[null_studio_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_studio['id'].max()
    if max_id > 0:
        df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_studio.head())

# Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_studio.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


     id                  Studio
0  1002            Toei Company
1     1      Universal Pictures
2     2  Chi-Fou-Mi Productions
3     3     Du Maurier-Guinness
4     4           Virgin Vision
     id  Numéro
0  1002    3721
1  1002    3716
2  1002    3681
3  1002    3639
4  1002    3565


3724

In [74]:
# df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

In [75]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3724

# table movie, casting & intermediaire

In [76]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [77]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [78]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [79]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [80]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [81]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [82]:
display(df_casting)

,id,Distribution
0,0,Shin'ichirô Hayashi
1,1,Reiko Ike
2,2,Miki Sugimoto
3,3,Yayoi Watanabe
4,4,Tōru Abe
...,...,...
10151,10151,Paulette Goddard
10152,10152,Henry Bergman
10153,10153,Tiny Sandford
10154,10154,Jackie Coogan


In [83]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3721,The Lustful Shogun and his 21 Concubines (vostfr),エロ将軍と二十一人の愛妾 (Titre original),1972,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le 5 octobre 1773 naissent au même moment Toyo...,F:\#DIVX\DIVX MOVIEZ\DRAMA\The Lustful Shogun ...,Matroska (MKV),NaN,Fichier multimédia,1.74 GB,movie/126896,2098680.0
1,3720,Les Indomptés,Mobsters (Titre original),1991,104,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IVRHM-BHd4A,Ils n'ont pas reçu d'ordres... Ils ont pris la...,"Dans les années 1910, deux gangs du milieu new...",F:\#DIVX\DIVX MOVIEZ\POLAR\Les Indomptés [UNCU...,Matroska (MKV),NaN,Fichier multimédia,2.57 GB,movie/21219,102460.0
2,3719,Le Deuxième Acte,NaN,2024,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=7aiYYA7o6_0,NaN,"Florence veut présenter David, l'homme dont el...",F:\#DIVX\DIVX MOVIEZ\COMEDIE\Le Deuxieme Acte ...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/1161108,30784256.0
3,3718,Le Bouc émissaire,The Scapegoat (Titre original),1959,92,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=knHLALAknFo,He took another man's name... lived another ma...,"Un maître d'école anglais rencontre son sosie,...",F:\#DIVX\DIVX MOVIEZ\POLAR\Le Bouc Emissaire -...,Matroska (MKV),NaN,Fichier multimédia,2.31 GB,movie/126919,53247.0
4,3717,Gothic (vostfr),NaN,1986,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=J_JQ8o2M68w,"Conjure up your deepest, darkest fear... now c...","Le 16 juin 1816 en Suisse, la nuit la plus cel...",F:\#DIVX\DIVX MOVIEZ\MadMov\Gothic (1986) VOST...,Matroska (MKV),NaN,Fichier multimédia,1.05 GB,movie/33516,91142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3717,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3718,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3719,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [84]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3721
1,1,3721
2,1,3508
3,1,3488
4,1,3439
...,...,...
17674,10151,2
17675,10152,2
17676,10153,2
17677,10154,1


In [85]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


      id         Distribution
0  10156  Shin'ichirô Hayashi
1      1            Reiko Ike
2      2        Miki Sugimoto
3      3       Yayoi Watanabe
4      4             Tōru Abe
      id  Numéro
0  10156    3721
1      1    3721
2      1    3508
3      1    3488
4      1    3439


17679

In [86]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [87]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [88]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [89]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [90]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [91]:
df_country_movie  = df_country_movie.explode("Pays")

In [92]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [93]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [94]:
display(df_country)

,id,Pays
0,0,Japon
1,1,Etats-Unis
2,2,France
3,3,Royaume-Uni
4,4,Corée du Sud
5,5,Inde
6,6,Italie
7,7,Canada
8,8,Belgique
9,9,Pays-Bas


In [95]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3721,The Lustful Shogun and his 21 Concubines (vostfr),エロ将軍と二十一人の愛妾 (Titre original),1972,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le 5 octobre 1773 naissent au même moment Toyo...,F:\#DIVX\DIVX MOVIEZ\DRAMA\The Lustful Shogun ...,Matroska (MKV),NaN,Fichier multimédia,1.74 GB,movie/126896,2098680.0
1,3720,Les Indomptés,Mobsters (Titre original),1991,104,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IVRHM-BHd4A,Ils n'ont pas reçu d'ordres... Ils ont pris la...,"Dans les années 1910, deux gangs du milieu new...",F:\#DIVX\DIVX MOVIEZ\POLAR\Les Indomptés [UNCU...,Matroska (MKV),NaN,Fichier multimédia,2.57 GB,movie/21219,102460.0
2,3719,Le Deuxième Acte,NaN,2024,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=7aiYYA7o6_0,NaN,"Florence veut présenter David, l'homme dont el...",F:\#DIVX\DIVX MOVIEZ\COMEDIE\Le Deuxieme Acte ...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/1161108,30784256.0
3,3718,Le Bouc émissaire,The Scapegoat (Titre original),1959,92,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=knHLALAknFo,He took another man's name... lived another ma...,"Un maître d'école anglais rencontre son sosie,...",F:\#DIVX\DIVX MOVIEZ\POLAR\Le Bouc Emissaire -...,Matroska (MKV),NaN,Fichier multimédia,2.31 GB,movie/126919,53247.0
4,3717,Gothic (vostfr),NaN,1986,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=J_JQ8o2M68w,"Conjure up your deepest, darkest fear... now c...","Le 16 juin 1816 en Suisse, la nuit la plus cel...",F:\#DIVX\DIVX MOVIEZ\MadMov\Gothic (1986) VOST...,Matroska (MKV),NaN,Fichier multimédia,1.05 GB,movie/33516,91142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3717,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3718,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3719,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [96]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3721
1,0,3716
2,0,3714
3,0,3698
4,0,3686
...,...,...
3716,42,853
3717,43,921
3718,44,769
3719,45,631


In [97]:
# Trouver l'index de la ligne avec le 'Pays' NULL
null_country_index = df_country[df_country['Pays'].isnull()].index

# Si une ligne avec 'Pays' NULL est trouvée
if not null_country_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_country.loc[null_country_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_country['id'].max()
    if max_id > 0:
        df_country.loc[df_country.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_country.head())

# Enregistrer le DataFrame modifié dans la table 'country' de la base de données
df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_country.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

   id          Pays
0  46         Japon
1   1    Etats-Unis
2   2        France
3   3   Royaume-Uni
4   4  Corée du Sud
   id  Numéro
0  46    3721
1  46    3716
2  46    3714
3  46    3698
4  46    3686


3721

In [98]:
# df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

In [99]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3721

# table movie, genre & intermediaire

In [100]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [101]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [102]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [103]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [104]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [105]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [106]:
display(df_genre)

,id,Genre(s)
0,0,Comédie
1,1,Adulte
2,2,Asia
3,3,Action
4,4,Suspense
5,5,Policier
6,6,Mystère
7,7,Horreur
8,8,Guerre
9,9,Chanbara


In [107]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3721,The Lustful Shogun and his 21 Concubines (vostfr),エロ将軍と二十一人の愛妾 (Titre original),1972,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le 5 octobre 1773 naissent au même moment Toyo...,F:\#DIVX\DIVX MOVIEZ\DRAMA\The Lustful Shogun ...,Matroska (MKV),NaN,Fichier multimédia,1.74 GB,movie/126896,2098680.0
1,3720,Les Indomptés,Mobsters (Titre original),1991,104,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IVRHM-BHd4A,Ils n'ont pas reçu d'ordres... Ils ont pris la...,"Dans les années 1910, deux gangs du milieu new...",F:\#DIVX\DIVX MOVIEZ\POLAR\Les Indomptés [UNCU...,Matroska (MKV),NaN,Fichier multimédia,2.57 GB,movie/21219,102460.0
2,3719,Le Deuxième Acte,NaN,2024,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=7aiYYA7o6_0,NaN,"Florence veut présenter David, l'homme dont el...",F:\#DIVX\DIVX MOVIEZ\COMEDIE\Le Deuxieme Acte ...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/1161108,30784256.0
3,3718,Le Bouc émissaire,The Scapegoat (Titre original),1959,92,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=knHLALAknFo,He took another man's name... lived another ma...,"Un maître d'école anglais rencontre son sosie,...",F:\#DIVX\DIVX MOVIEZ\POLAR\Le Bouc Emissaire -...,Matroska (MKV),NaN,Fichier multimédia,2.31 GB,movie/126919,53247.0
4,3717,Gothic (vostfr),NaN,1986,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=J_JQ8o2M68w,"Conjure up your deepest, darkest fear... now c...","Le 16 juin 1816 en Suisse, la nuit la plus cel...",F:\#DIVX\DIVX MOVIEZ\MadMov\Gothic (1986) VOST...,Matroska (MKV),NaN,Fichier multimédia,1.05 GB,movie/33516,91142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3717,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3718,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3719,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [108]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3721
1,0,3719
2,0,3714
3,0,3711
4,0,3710
...,...,...
8185,37,36
8186,37,35
8187,38,3113
8188,38,389


In [109]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id  Genre(s)
0  38   Comédie
1   1    Adulte
2   2      Asia
3   3    Action
4   4  Suspense
   id  Numéro
0  38    3721
1  38    3719
2  38    3714
3  38    3711
4  38    3710


8190

In [110]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [111]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

8190

# table movie, language & intermediaire

In [112]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [113]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [114]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [115]:
df_language_movie  = df_language_movie.explode("Langues")

In [116]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [117]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [118]:
display(df_language)

,id,Langues
0,0,Japonais
1,1,Anglais
2,2,Espagnol
3,3,Allemand
4,4,Français
5,5,NaN
6,6,Coréen
7,7,Italien
8,8,Néerlandais
9,9,Polonais


In [119]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3721,The Lustful Shogun and his 21 Concubines (vostfr),エロ将軍と二十一人の愛妾 (Titre original),1972,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le 5 octobre 1773 naissent au même moment Toyo...,F:\#DIVX\DIVX MOVIEZ\DRAMA\The Lustful Shogun ...,Matroska (MKV),NaN,Fichier multimédia,1.74 GB,movie/126896,2098680.0
1,3720,Les Indomptés,Mobsters (Titre original),1991,104,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IVRHM-BHd4A,Ils n'ont pas reçu d'ordres... Ils ont pris la...,"Dans les années 1910, deux gangs du milieu new...",F:\#DIVX\DIVX MOVIEZ\POLAR\Les Indomptés [UNCU...,Matroska (MKV),NaN,Fichier multimédia,2.57 GB,movie/21219,102460.0
2,3719,Le Deuxième Acte,NaN,2024,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=7aiYYA7o6_0,NaN,"Florence veut présenter David, l'homme dont el...",F:\#DIVX\DIVX MOVIEZ\COMEDIE\Le Deuxieme Acte ...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/1161108,30784256.0
3,3718,Le Bouc émissaire,The Scapegoat (Titre original),1959,92,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=knHLALAknFo,He took another man's name... lived another ma...,"Un maître d'école anglais rencontre son sosie,...",F:\#DIVX\DIVX MOVIEZ\POLAR\Le Bouc Emissaire -...,Matroska (MKV),NaN,Fichier multimédia,2.31 GB,movie/126919,53247.0
4,3717,Gothic (vostfr),NaN,1986,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=J_JQ8o2M68w,"Conjure up your deepest, darkest fear... now c...","Le 16 juin 1816 en Suisse, la nuit la plus cel...",F:\#DIVX\DIVX MOVIEZ\MadMov\Gothic (1986) VOST...,Matroska (MKV),NaN,Fichier multimédia,1.05 GB,movie/33516,91142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3717,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3718,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3719,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [120]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3721
1,0,3716
2,0,3714
3,0,3698
4,0,3686
...,...,...
5229,42,1060
5230,43,1053
5231,43,930
5232,43,926


In [121]:
# Trouver l'index de la ligne avec les 'Langues' NULL
null_language_index = df_language[df_language['Langues'].isnull()].index

# Si une ligne avec 'Langues' NULL est trouvée
if not null_language_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_language.loc[null_language_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_language['id'].max()
    if max_id > 0:
        df_language.loc[df_language.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_language.head())

# Enregistrer le DataFrame modifié dans la table 'language' de la base de données
df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_language.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


   id   Langues
0  45  Japonais
1   1   Anglais
2   2  Espagnol
3   3  Allemand
4   4  Français
   id  Numéro
0  45    3721
1  45    3716
2  45    3714
3  45    3698
4  45    3686


5234

In [122]:
# df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

In [123]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

5234

# table, movie, tag & intermediaire

In [124]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [125]:
list_tag_movie = ["Etiquettes", 'Numéro']
df_tag_movie = df.copy()[list_tag_movie]

In [126]:
df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [127]:
df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [128]:
df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [129]:
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [130]:
display(df_tag)

,id,Etiquettes
0,0,NaN
1,1,Yakusa
2,2,Chanbara
3,3,Sport
4,4,Ange
...,...,...
203,203,Momie
204,204,viking
205,205,Réalisme soviétique
206,206,Guy Debord


In [131]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3721,The Lustful Shogun and his 21 Concubines (vostfr),エロ将軍と二十一人の愛妾 (Titre original),1972,95,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Le 5 octobre 1773 naissent au même moment Toyo...,F:\#DIVX\DIVX MOVIEZ\DRAMA\The Lustful Shogun ...,Matroska (MKV),NaN,Fichier multimédia,1.74 GB,movie/126896,2098680.0
1,3720,Les Indomptés,Mobsters (Titre original),1991,104,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=IVRHM-BHd4A,Ils n'ont pas reçu d'ordres... Ils ont pris la...,"Dans les années 1910, deux gangs du milieu new...",F:\#DIVX\DIVX MOVIEZ\POLAR\Les Indomptés [UNCU...,Matroska (MKV),NaN,Fichier multimédia,2.57 GB,movie/21219,102460.0
2,3719,Le Deuxième Acte,NaN,2024,85,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=7aiYYA7o6_0,NaN,"Florence veut présenter David, l'homme dont el...",F:\#DIVX\DIVX MOVIEZ\COMEDIE\Le Deuxieme Acte ...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/1161108,30784256.0
3,3718,Le Bouc émissaire,The Scapegoat (Titre original),1959,92,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=knHLALAknFo,He took another man's name... lived another ma...,"Un maître d'école anglais rencontre son sosie,...",F:\#DIVX\DIVX MOVIEZ\POLAR\Le Bouc Emissaire -...,Matroska (MKV),NaN,Fichier multimédia,2.31 GB,movie/126919,53247.0
4,3717,Gothic (vostfr),NaN,1986,84,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=J_JQ8o2M68w,"Conjure up your deepest, darkest fear... now c...","Le 16 juin 1816 en Suisse, la nuit la plus cel...",F:\#DIVX\DIVX MOVIEZ\MadMov\Gothic (1986) VOST...,Matroska (MKV),NaN,Fichier multimédia,1.05 GB,movie/33516,91142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3717,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3718,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3719,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [132]:
display(df_intermediaire_movie_tag)

,id,Numéro
0,0,3721
1,0,3720
2,0,3719
3,0,3718
4,0,3717
...,...,...
4678,206,74
4679,206,73
4680,207,17
4681,207,16


In [133]:
# Trouver l'index de la ligne avec les 'Etiquettes' NULL
null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# Si une ligne avec 'Etiquettes' NULL est trouvée
if not null_tag_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_tag.loc[null_tag_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_tag['id'].max()
    if max_id > 0:
        df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_tag.head())

# Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_tag.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


    id Etiquettes
0  208        NaN
1    1     Yakusa
2    2   Chanbara
3    3      Sport
4    4       Ange
    id  Numéro
0  208    3721
1  208    3720
2  208    3719
3  208    3718
4  208    3717


4683

In [134]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [135]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )